In [ ]:
import http.server
import socket
import socketserver
import webbrowser
import pyqrcode
import os
import subprocess
import time
import requests
import getpass
from urllib.parse import unquote

PORT = 8010
TEMP_FILE = "shared_text.txt"

def authenticate_user():
    try:
        import biometric_auth  # Placeholder for actual biometric auth module
        if biometric_auth.verify():  # Assuming a function that verifies biometric identity
            print("Biometric authentication successful.")
            return True
        else:
            print("Biometric authentication failed.")
            return False
    except ImportError:
        print("Biometric authentication module not found. Falling back to password authentication.")
        password = getpass.getpass("Enter access password: ")
        return password == "securepass"  # Replace with a hashed or more secure verification

choice = input("Do you want to share a text message or a file? (text/file): ").strip().lower()

if choice == "text":
    text_content = input("Enter the text message to share: ")
    with open(TEMP_FILE, "w", encoding="utf-8") as f:
        f.write(text_content)
    file_to_serve = TEMP_FILE
    directory_to_serve = os.getcwd()

elif choice == "file":
    file_path = input("Enter the file path to serve: ").strip('"')
    if not os.path.isfile(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        exit(1)
    directory_to_serve = os.path.dirname(file_path)
    file_to_serve = os.path.basename(file_path)
else:
    print("Invalid choice. Please enter 'text' or 'file'.")
    exit(1)

os.chdir(directory_to_serve)

class CustomHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        if not authenticate_user():
            self.send_error(403, "Unauthorized access - Biometric authentication failed")
            return
        requested_path = unquote(self.path.lstrip("/"))
        if requested_path != file_to_serve:
            self.send_error(404, "File not found")
        else:
            self.path = f"/{file_to_serve}"
            return super().do_GET()

try:
    print("Starting ngrok... Please wait.")
    ngrok_process = subprocess.Popen(
        [r"C:\ngrok.exe", "http", str(PORT)],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    time.sleep(5)
    
    ngrok_url = None
    try:
        # response = requests.get("http://127.0.0.1:4040/api/tunnels")
        response = requests.get("http://192.168.29.14:8010/api/tunnels")
        ngrok_data = response.json()
        ngrok_url = ngrok_data["tunnels"][0]["public_url"]
    except Exception as e:
        print(f"Error fetching ngrok URL: {e}")
    
    if ngrok_url:
        file_url = f"{ngrok_url}/{file_to_serve}"
        qr_code = pyqrcode.create(file_url, encoding='utf-8')
        qr_code.png("myqr.png", scale=8)
        webbrowser.open("myqr.png")
    else:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        try:
            s.connect(("8.8.8.8", 80))
            local_ip = s.getsockname()[0]
        except:
            local_ip = "127.0.0.1"
        finally:
            s.close()
        local_url = f"http://{local_ip}:{PORT}/{file_to_serve}"
        qr_code = pyqrcode.create(local_url)
        qr_code.png("myqr.png", scale=8)
        webbrowser.open("myqr.png")
    
    with socketserver.TCPServer(("0.0.0.0", PORT), CustomHandler) as httpd:
        print(f"Serving '{file_to_serve}' at port {PORT}")
        print("Scan the QR code to access the file.")
        httpd.serve_forever()

except KeyboardInterrupt:
    print("\nShutting down server.")
finally:
    if ngrok_process:
        ngrok_process.terminate()
    if choice == "text" and os.path.exists(TEMP_FILE):
        os.remove(TEMP_FILE)